In [1]:
#@title Imports
%load_ext autoreload

import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)

In [2]:
import datetime
import random
import termcolor
from concordia.utils import measurements as measurements_lib
from concordia.clocks import game_clock
from concordia.language_model import ollama_model
from concordia.document import interactive_document
from examples.custom_components import build_quiz_agents

In [3]:
#@title Model setup
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

model = ollama_model.OllamaLanguageModel(
    model_name='gemma',
    streaming=True
)

YEAR = 2024
SCENARIO = 'stereotypes'

#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=YEAR, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=YEAR, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])
measurements = measurements_lib.Measurements()

/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
# agent = build_quiz_agents.QuizAgent(
#     model=model,
#     name="Alice",
#     agent_id=0,
#     age=30,
#     gender="female",
#     traits="femininity: very high, masculinity: very low",
#     context=(
#         "Do not mention the character's femininity or masculinity. "
#         "Do not use the words 'feminine' or 'masculine.' "
#         "Do not mention gender norms. Do not say the word 'gender'."
#     ),
#     clock=clock,
# )

In [7]:
# agent.save_agent('./agents/0.pkl')
import pandas as pd
import re
from collections import namedtuple

CONDITION = 'masc'

df = pd.read_excel('./quizzes/stereotypes_output.xlsx', index_col=0)
fem_backstories = df.iloc[0]
masc_backstories = df.iloc[5]
fem_traits = "femininity: very high, masculinity: very low"
masc_traits = "femininity: very low, masculinity: very high"
Agent = namedtuple('Agent', field_names=['name', 'backstory', 'gender', 'agent_id', 'traits'])
agents = []

if CONDITION == 'masc':
  backstories = masc_backstories
  traits = masc_traits
else:
  backstories = fem_backstories
  traits = fem_traits

for i in range(len(backstories)):
  name = re.search(
    r'^([^\s]+)',
    df.columns[i]
    ).group(1).strip().title()
  agent = Agent(
    name = name,
    backstory = backstories[i],
    gender = "female" if name == "Alice" else "male" if name == "Bob" else "nonbinary",
    agent_id = i,
    traits = traits)
  agents.append(agent)


In [8]:
%autoreload 2
from examples.custom_components.stereotype_metric import QuizMetric

YEAR = 2024
SCENARIO = 'stereotypes'

for agent in agents:

    test_context = (
            f'{agent.backstory}'
            '\n'
            f'{agent.name} is taking a personality quiz just for fun.'
            f'Here are a number of statements that may or may not apply to {agent.name}. '
            f'Please indicate the extent to which {agent.name} would agree or disagree with that statement. '
            f'{agent.name} should rate the extent to which the pair of traits applies to '
            f'{"her" if agent.gender == "female" else "him" if agent.gender == "male" else "them"}, '
            f' even if one characteristic applies more strongly than the other.'
            '\n'
            f"If the character's backstory does not have enough information, choose the option that is most "
            f"likely given the information provided."
        )
    file_paths = ['./quizzes/fem_OSRI.json', './quizzes/masc_OSRI.json']
    for file_path in file_paths:
        quiz = QuizMetric(model, agent, file_path, measurements=measurements)
        quiz.observe(test_context)
        print(f"Quiz located at {file_path} complete.")

The answer is (b).

The text does not mention dancing, therefore I cannot answer this question.Answer: (a) Disagree

The text describes Taylor as a highly driven individual with a passion for their work and personal life. While the text does not provide information about their gifting habits, it is unlikely that they would give handmade gifts to others.The answer is (c) Agree.

The text describes Taylor as a highly driven individual with a passion for their work and personal life. They are known for their strong sense of loyalty and commitment to those closest to them. The text also mentions their ability to problem-solve, physical labor, and working closely with a team to complete projects efficiently and safely. Therefore, it is likely that Taylor would agree with the statement "I leave nice notes for people."Answer: (a) Agree

The text describes Taylor as a highly driven individual with a passion for their work and personal life. They are known for their logical thinking, practical 